In [ ]:
import cv2
import dlib
import numpy as np

# DNNベースの顔検出器のロード
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# カメラの指定
cap = cv2.VideoCapture(0)

# 肌のスムージング用のぼかしフィルタの強さ
smooth_strength = 25

# 目の拡大比率
eye_scale = 1.2

# 小顔効果の縮小率
face_shrink_ratio = 0.9

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # グレースケール画像に変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顔を検出
    faces = face_detector(gray)
    
    for face in faces:
        # 顔のランドマークを取得
        landmarks = landmark_predictor(gray, face)

        # --- 美顔補正処理 ---

        # 1. 肌のスムージング (スキンソフトニング)
        face_rect = frame[face.top():face.bottom(), face.left():face.right()]
        smoothed_face = cv2.bilateralFilter(face_rect, d=9, sigmaColor=smooth_strength, sigmaSpace=smooth_strength)
        frame[face.top():face.bottom(), face.left():face.right()] = smoothed_face

        # 2. 小顔効果の適用（未変更）

        # 3. 目を大きくする処理
        left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
        right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])

        # 左目の拡大処理
        left_eye_rect = cv2.boundingRect(left_eye_points)
        x, y, w, h = left_eye_rect
        left_eye_roi = frame[y:y + h, x:x + w]

        # 左目のアフィン変換（拡大）
        src_points = np.float32([[0, 0], [w, 0], [0, h]])
        dst_points = np.float32([[0, 0], [int(w * eye_scale), 0], [0, int(h * eye_scale)]])
        M = cv2.getAffineTransform(src_points, dst_points)
        big_left_eye = cv2.warpAffine(left_eye_roi, M, (int(w * eye_scale), int(h * eye_scale)))

        # 拡大された目を元の位置に収める
        frame[y:y + big_left_eye.shape[0], x:x + big_left_eye.shape[1]] = big_left_eye

        # 右目の拡大処理
        right_eye_rect = cv2.boundingRect(right_eye_points)
        x, y, w, h = right_eye_rect
        right_eye_roi = frame[y:y + h, x:x + w]

        # 右目のアフィン変換（拡大）
        src_points = np.float32([[0, 0], [w, 0], [0, h]])
        dst_points = np.float32([[0, 0], [int(w * eye_scale), 0], [0, int(h * eye_scale)]])
        M = cv2.getAffineTransform(src_points, dst_points)
        big_right_eye = cv2.warpAffine(right_eye_roi, M, (int(w * eye_scale), int(h * eye_scale)))

        # 拡大された目を元の位置に収める
        frame[y:y + big_right_eye.shape[0], x:x + big_right_eye.shape[1]] = big_right_eye

        # 4. 明るさとコントラストの調整
        frame = cv2.convertScaleAbs(frame, alpha=1.2, beta=20)  # 明るさとコントラストを調整

    # フレームを表示
    cv2.imshow('Beauty Filter with Small Face Effect', frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 終了処理
cap.release()
cv2.destroyAllWindows()
